# Book Recommendation Engine using KNN

Create a book recommendation algorithm using **K-Nearest Neighbors** (algorithm measures the distance to determine the “closeness” of instances).

You will use the [Book-Crossings dataset](https://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users.

function `get_recommends()` takes book title (from the dataset) as an argument and returns list of 5 similar books with distances

code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

returns:

```python
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301],
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479],
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-10-05 00:58:27--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2023-10-05 00:58:27 (193 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# merge both dataframes into one dataframe
df_merged = pd.merge(df_books, df_ratings, on='isbn', how="right")

# ensure statistical significance, remove users with less than 200 ratings
user_above_200 = df_merged['user'].value_counts(ascending=True).to_frame().query("user >= 200")

# ensure statistical significance, remove books with less than 100 ratings
isbn_above_100 = df_merged['isbn'].value_counts(ascending=True).to_frame().query("isbn >= 100")

# ensure rows where only statistical significance conditions are met
df_merged_filtered = df_merged[(df_merged['user'].isin(user_above_200.index)) & (df_merged['isbn'].isin(isbn_above_100.index))]

# drop rows where all values are missing
df_merged_filtered.dropna(inplace=True, how='all')

# drop duplicates based on same title and user
df_merged_filtered.drop_duplicates(subset=['title', 'user'], keep='first', inplace=True)

# drop duplicates based on same title, user and author
df_merged_filtered.drop_duplicates(subset=['title', 'user', 'author'], keep='first', inplace=True)

<ipython-input-4-6a056d1e9386>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_filtered.dropna(inplace=True, how='all')
<ipython-input-4-6a056d1e9386>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_filtered.drop_duplicates(subset=['title', 'user'], keep='first', inplace=True)
<ipython-input-4-6a056d1e9386>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_filtered.drop_duplicates(subset=['title', 'use

In [5]:
# generate pivot table
df_pivot = df_merged_filtered.pivot_table(index = 'title', columns = 'user', values = 'rating').fillna(0)

# generate sparse matrix
csr_matrix = csr_matrix(df_pivot)

# generate nearest neighbors model
nn = NearestNeighbors(metric='cosine').fit(csr_matrix)

In [6]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  # get distance and indices given book
  distance, indices = nn.kneighbors([df_pivot.loc[book]], n_neighbors=6, return_distance=True)

  # get book list, flatten array into 1d, drop first element, reverse entire array and pull out index only
  book_list = df_pivot.iloc[indices.flatten()[1:][::-1]].index.tolist()

  # turn 2d array into 1d array, drop first element, reverse entire array
  distance_filtered_reversed = distance.flatten()[1:][::-1]

  # generate recommendation matrix
  recommend_matrix = [[book_list[i], distance_filtered_reversed[i]] for i in range(len(book_list))]

  # generate recommended books matrix
  recommended_books = [
    df_pivot.iloc[indices.flatten()[0]].name,
    recommend_matrix
    ]

  return recommended_books

In [8]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
